<a href="https://colab.research.google.com/github/duncan60/llm-notes/blob/main/langchain_LLM_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用 langchain 實現 LLM + RAG 入門應用
1. [LangChain](https://python.langchain.com/v0.2/docs/introduction/)
- 定義：LangChain是一個框架，用於構建基於語言模型的應用。
- 功能：支持多種語言模型的集成，提供一系列工具和庫來簡化開發過程。
2. RAG（Retrieval-Augmented Generation 檢索增強生成）
- 定義：RAG是一種將檢索技術與生成技術結合的方法，用於提高語言生成模型的性能。
- 運作原理：先從大量數據中檢索相關信息，然後根據檢索結果生成回答。
3. VectorDB: [supabase](https://supabase.com/)
- 定義：專門設計來處理高維度向量數據。這些向量數據通常來自於各種機器學習和深度學習模型的特徵表示。
- 功能：提供高效的向量檢索、向量存儲、數據管理


**結合的原因及優勢**
- 原因：LangChain提供了強大的集成能力和靈活的開發框架，RAG則能提升生成模型的準確性和實用性。
- 優勢：結合LangChain和RAG可以更高效地處理大型數據集，生成更加準確和相關的回答。


---


學習資源：
- https://medium.com/@cch.chichieh/rag%E5%AF%A6%E4%BD%9C%E6%95%99%E5%AD%B8-streamlit-langchain-llama2-c7d1dac2494e
- https://python.langchain.com/v0.2/docs/tutorials/rag/
- https://www.youtube.com/watch?v=d60FAktet4M
- https://www.youtube.com/watch?v=lWW6ZCBKg1I
- https://www.youtube.com/watch?v=NM1jPPGwNtk


## Step1: 安裝所需要套件

### 主要套件說明
1. langchain: 一個用於構建大型語言模型 (LLM) 應用程式的框架。
2. langchain_community: 由社群貢獻的 LangChain 整合與擴充套件集合。
3. langchain_openai: 專為與 OpenAI 語言模型整合而設計。
4. supabase: 一個開源的 Firebase 替代方案，用於構建 Web 和行動應用程式。


In [ ]:
!pip install --upgrade --quiet langchain langchain_community langchain_openai tiktoken supabase unstructured

## Step 2: 引用相關設定與初始化

In [ ]:
import os
from google.colab import userdata
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from supabase.client import Client, create_client

os.environ["SUPABASE_URL"] = userdata.get("SUPABASE_URL")
os.environ["SUPABASE_SERVICE_KEY"] = userdata.get("SUPABASE_SERVICE_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")

supbase_url = os.environ["SUPABASE_URL"]
supbase_key = os.environ["SUPABASE_SERVICE_KEY"]

supabase_client: Client = create_client(supbase_url, supbase_key)
embeddings = OpenAIEmbeddings()

## Step3: 讀取內容切格文檔

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader

urls = ["https://github.com/qiangmzsx/Software-Engineering-at-Google/blob/main/zh-cn/Chapter-1_What_Is_Software_Engineering/Chapter-1_What_Is_Software_Engineering.md"]

loader = UnstructuredURLLoader(urls=urls)
docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

104

In [ ]:
splits

## Step4: 存入 supabase DB

- langchain 使用介紹：https://python.langchain.com/v0.2/docs/integrations/vectorstores/supabase/
- 透過 Supabase SQL Editor ，從Quickstarts 挑選 langchain ，建立 DB chema

In [ ]:
vectorstore = SupabaseVectorStore.from_documents(
    splits,
    embeddings,
    client=supabase_client,
    table_name="documents",
    query_name="match_documents"
)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("軟體工程師與工程師兩者之間的差異是什麼")

In [ ]:
retrieved_docs

## Step5: RAG + LLM 結合效果
從向量資料找出關聯片段，與問題組合成 prompt 給大語言模型

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = '''
您是問答任務的助手，請使用以下檢索到的上下文來回答問題。如果你不知道答案或問題不是檢索範圍內的內容，就說無法回答該問題，不要試圖編造答案。使用繁體中文回答
Question: {question}
Context: {context}
Answer:
'''

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = ({"context": (retriever | format_docs), "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(prompt)
    | ChatOpenAI(model="gpt-4o")
    | StrOutputParser())



In [ ]:
rag_chain.invoke("軟體工程師與工程師兩者之間的差異是什麼，請用列表比較兩者間的差異？")

'軟體工程師與工程師之間的差異可以通過以下列表來比較：\n\n1. **時間**：\n   - **軟體工程師**：更關注長期的時間成本和需求變更，考慮到軟體的維護和持續使用。\n   - **工程師**：專注於當前的開發任務，可能不會考慮長期的維護和擴展。\n\n2. **規模**：\n   - **軟體工程師**：通常涉及多人協作，處理多版本的程序，注重團隊合作和管理。\n   - **工程師**：通常是個人的創造行為，規模較小，協作需求較少。\n\n3. **決策複雜性與風險**：\n   - **軟體工程師**：需要在多個路徑之間做出權衡，風險較高，決策複雜，需考慮長期的擴展性和可持續性。\n   - **工程師**：決策相對簡單，主要集中在當前問題的解決上，風險較低。\n\n4. **開發範疇**：\n   - **軟體工程師**：包括開發、修改和維護，使用策略、實踐和工具來確保代碼在長期內有效。\n   - **工程師**：主要集中在代碼的生成，即開發階段。\n\n5. **團隊協作**：\n   - **軟體工程師**：通常在團隊環境中工作，需要協作和管理多人的貢獻。\n   - **工程師**：多數是個人獨立完成開發工作，協作需求較少。\n\n6. **最佳實踐**：\n   - **軟體工程師**：依賴於集成測試、持續部署（CD）、語義版本控制（SemVer）和依賴性管理等長期實踐。\n   - **工程師**：對於短期項目，可以使用任何可用的手段來解決當前任務，無需依賴上述長期實踐。\n\n總結來說，軟體工程師和工程師在時間維度、規模、決策複雜性、開發範疇、團隊協作和最佳實踐等方面存在顯著差異。'

In [ ]:
rag_chain.invoke("汽車與飛機兩者之間的差異是什麼，請用列表比較兩者間的差異？")

'無法回答該問題。您提供的上下文並未涉及汽車與飛機之間的差異。'